# Your Title Here

**Name(s)**: Landon Marchetti and Lyn Mansfield

**Website Link**: (your website link)

In [15]:
import pandas as pd
import numpy as np
from pathlib import Path
from matplotlib.backends.backend_pdf import PdfPages

import plotly.express as px
pd.options.plotting.backend = 'plotly'

from dsc80_utils import * # Feel free to uncomment and use this.

## Step 1: Introduction

# TODO

Which legends are the most meta-defining? Difference in win rate between when it's banned vs when it's not

How important is the first kill of a game? 

Which characters are the most banned? Which characters are the best?

What archetype of character's performance matters the most?

## Step 2: Data Cleaning and Exploratory Data Analysis

# TODO
What are "partial" rows missing from their data? <== me!

In [36]:
# TODO

from pathlib import Path

dataset_path = Path("./data/2025_LoL_esports_match_data_from_OraclesElixir.csv")
# /data/2025_LoL_esports_match_data_from_OraclesElixir.csv
test_s = open(dataset_path).read()
rows = test_s.split('\n')
columnss = rows[0].split(',')
rowss = [x.split(',') for x in rows[1:]]
data = pd.DataFrame(columns = columnss, data = rowss)

In [37]:
sample = data.iloc[:192]
sample['gamelength'].iloc[0]

'1592'

## Step 3: Assessment of Missingness

In [16]:
# TODO

def univariate_plots(df, title):
    pdf_pages = PdfPages(f'univariate_plots_{title}.pdf')

    plt.figure()

    for column in df.columns:
        fig, ax = plt.subplots()
        ax.hist(df[column])
        ax.set_title(f'Graph of {column}')
        pdf_pages.savefig(fig)
        plt.close(fig)
    
    pdf_pages.close()

def bivariate_plot(df, categorical_column, target_column, title):
    pdf_pages = PdfPage(f'bivariate_plots_{title}.pdf')

    plt.figure()
    
    for column in df.columns:
        fig, ax = plt.subplots()
        ax.hist(df[column])
        ax.set_title(f'Graph of {column}')
        pdf_pages.savefig(fig)
        plt.close(fig)
    
    pdf_pages.close()

In [17]:
df = pd.DataFrame({
	'A': np.random.uniform(0, 10, 100), 
	'B': np.random.uniform(0, 10, 100), 
	'C': np.random.uniform(0, 10, 100)
	})
univariate_plots(df, 'test')

<Figure size 1000x500 with 0 Axes>

## Step 4: Hypothesis Testing

In [ ]:
# TODO

## Step 5: Framing a Prediction Problem

In [ ]:
# TODO

## Step 6: Baseline Model

In [ ]:
# TODO

## Step 7: Final Model

In [ ]:
# TODO

## Step 8: Fairness Analysis

In [ ]:
# TODO